In [ ]:
import os
import cv2
import random
from tqdm import tqdm

# Paths
source_dataset = r"E:\New folder\isl_dataset-cropped"  # Your cropped dataset
target_dataset = r"E:\New folder\isl_brightness_contrast"  # Augmented dataset output folder

if not os.path.exists(target_dataset):
    os.makedirs(target_dataset)

def random_brightness_contrast(image, brightness_range=(-40, 40), contrast_range=(0.9, 1.1)):
    """
    Apply random brightness and contrast adjustment.
    brightness_range: Range of brightness shift (-40 to +40)
    contrast_range: Range of contrast multiplier (0.9x to 1.1x)
    """
    brightness = random.randint(*brightness_range)
    contrast = random.uniform(*contrast_range)

    # Apply contrast and brightness
    img_aug = cv2.convertScaleAbs(image, alpha=contrast, beta=brightness)
    return img_aug

def apply_and_save(image_path, save_dir, image_name):
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Failed to load image: {image_path}")
        return False

    base, ext = os.path.splitext(image_name)

    # 1️⃣ Save Original Image
    orig_path = os.path.join(save_dir, f"{base}{ext}")
    cv2.imwrite(orig_path, img)

    # 2️⃣ Save Brightness/Contrast Augmented Image
    bc_aug_img = random_brightness_contrast(img)
    bc_aug_name = f"{base}_bc{ext}"
    bc_aug_path = os.path.join(save_dir, bc_aug_name)
    cv2.imwrite(bc_aug_path, bc_aug_img)

    return True

# Walk through dataset
for split in ["train", "valid", "test"]:
    split_dir = os.path.join(source_dataset, split)
    target_split_dir = os.path.join(target_dataset, split)

    for class_name in os.listdir(split_dir):
        class_dir = os.path.join(split_dir, class_name)
        target_class_dir = os.path.join(target_split_dir, class_name)
        os.makedirs(target_class_dir, exist_ok=True)

        image_files = [f for f in os.listdir(class_dir) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
        print(f"\n💡 Applying Brightness/Contrast: {split}/{class_name} ({len(image_files)} images)")

        for img_name in tqdm(image_files, desc=f"{split}/{class_name}"):
            src_img_path = os.path.join(class_dir, img_name)
            apply_and_save(src_img_path, target_class_dir, img_name)

print("\n✅ Brightness/Contrast augmentation complete. New dataset saved to:")
print(target_dataset)
